In [ ]:
import pygame
import serial
import time

# Configuração da Serial (Ajuste a porta COM conforme seu PC)
ser = serial.Serial('COM9', 115200, timeout=1) 
time.sleep(2) # Aguarda o Arduino resetar

pygame.init()
pygame.joystick.init()

if pygame.joystick.get_count() == 0:
    print("Controle de PS4 não encontrado!")
    exit()

joystick = pygame.joystick.Joystick(0)
joystick.init()

print(f"Controlando: {joystick.get_name()}")

def map_value(x, in_min, in_max, out_min, out_max):
    return int((x - in_min) * (out_max - out_min) / (in_max - in_min) + out_min)

try:
    while True:
        pygame.event.pump() # Atualiza os eventos do controle

        # Eixo 0: Esquerda/Direita (Pan) | Eixo 1: Cima/Baixo (Tilt)
        axis_x = joystick.get_axis(0)
        axis_y = joystick.get_axis(1)

        # Mapeia de -1.0/1.0 para 0/180 graus
        # Inverti o eixo Y para o "cima" ser valor maior, se preferir o contrário, troque 180 por 0
        pos1 = map_value(axis_x, -1.0, 1.0, 180,0 )
        pos2 = map_value(axis_y, -1.0, 1.0, 0, 180) 

        # Lógica simples para os LEDs usando botões (X = Azul, O = Verde)
        led = 0
        if joystick.get_button(0): led = 1 # Botão X
        if joystick.get_button(1): led = 2 # Botão Círculo
        if joystick.get_button(0) and joystick.get_button(1): led = 3

        # Monta a string no formato: "pos1,pos2,led\n"
        data_str = f"{pos1},{pos2},{led}\n"
        ser.write(data_str.encode())

        print(f"Enviado: {data_str.strip()}")
        
        time.sleep(0.05) # Delay para não inundar o buffer da serial

except KeyboardInterrupt:
    ser.close()
    pygame.quit()

Reconhecimento de face

In [ ]:
import cv2
import serial
import time
import numpy as np
import os

# --- Ajustes de Controle (Mecatrônica) ---
PORTA_SERIAL = 'COM9'
BAUD_RATE = 115200
ALPHA = 0.1      # Suavização (Quanto menor, mais lento)
DEADZONE = 10    # Zona morta em pixels
HOME_X, HOME_Y = 90, 30  # Posição de repouso (X=Centro, Y=Cima)

# Variáveis de persistência (Memória)
contador_perda = 0
LIMITE_PERDA = 15  # Quantos frames ele espera antes de voltar para o Home

# Variáveis de estado inicial
suave_x, suave_y = HOME_X, HOME_Y

try:
    if 'ser' in locals(): ser.close()
    ser = serial.Serial(PORTA_SERIAL, BAUD_RATE, timeout=0.1)
    time.sleep(2)
    print(f"Sistema pronto! Arduino em {PORTA_SERIAL}")
except Exception as e:
    print(f"Erro Serial: {e}")

# Carrega a inteligência (Haar Cascade)
xml_file = "haarcascade_frontalface_default.xml"
face_cascade = cv2.CascadeClassifier(xml_file)

url_esp = "http://192.168.0.214:81/stream" 
cap = cv2.VideoCapture(url_esp)

while True:
    ret, frame = cap.read()
    if not ret or frame is None:
        continue

    # 1. Processamento de Imagem
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    # Reduzir levemente o ruído ajuda no reconhecimento
    gray = cv2.equalizeHist(gray) 
    
    ih, iw = frame.shape[:2]
    centro_tela_x, centro_tela_y = iw // 2, ih // 2

    # Detecção mais sensível (scaleFactor menor = mais precisão)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=6, minSize=(45, 45))

    if len(faces) > 0:
        contador_perda = 0  # Resetamos a memória de perda
        
        # Pega o rosto mais próximo (maior área)
        (x, y, w, h) = sorted(faces, key=lambda f: f[2]*f[3], reverse=True)[0]
        
        cx = x + w // 2
        cy = y + h // 2

        # Monitoramento visual
        cv2.rectangle(frame, (x, y), (x + w, y + h), (0, 255, 0), 2)
        cv2.circle(frame, (cx, cy), 5, (0, 0, 255), -1)

        # 2. Lógica de Controle
        if abs(cx - centro_tela_x) > DEADZONE or abs(cy - centro_tela_y) > DEADZONE:
            alvo_x = int(np.interp(cx, [0, iw], [0, 180]))
            alvo_y = int(np.interp(cy, [0, ih], [0, 180]))

            suave_x = int(suave_x + ALPHA * (alvo_x - suave_x))
            suave_y = int(suave_y + ALPHA * (alvo_y - suave_y))

            ser.write(f"{suave_x},{suave_y},2\n".encode())
    else:
        # Lógica de Persistência: Ele só volta para o Home após X frames sem ninguém
        contador_perda += 1
        
        if contador_perda > LIMITE_PERDA:
            # Retorno suave para a posição HOME (Olhando para cima)
            suave_x = int(suave_x + ALPHA * (HOME_X - suave_x))
            suave_y = int(suave_y + ALPHA * (HOME_Y - suave_y))
            ser.write(f"{suave_x},{suave_y},1\n".encode())
        else:
            # Se perdeu por pouco tempo, mantém a última posição conhecida (LED Azul)
            ser.write(f"{suave_x},{suave_y},1\n".encode())

    # 3. Interface Visual
    cv2.line(frame, (centro_tela_x, 0), (centro_tela_x, ih), (255, 255, 255), 1)
    cv2.line(frame, (0, centro_tela_y), (iw, centro_tela_y), (255, 255, 255), 1)
    cv2.putText(frame, f"Perda: {contador_perda}", (10, 30), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 255, 255), 2)
    
    cv2.imshow('Monitor de Mecatronica - ESP32 Tracking', frame)

    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

cap.release()
cv2.destroyAllWindows()
ser.close()

Sistema pronto! Arduino em COM9
